In [2]:
from ultralytics import YOLO
import serial 
import torch
import cv2
import os
import io

In [3]:
model = YOLO("best.pt")

In [ ]:
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

ser = serial.Serial()
ser.baudrate = 9600
ser.port = 'COM5'
ser.open()

while True:
    _, img = cap.read() 


    #computes prediction
    results = model(img, verbose=False, conf=0.7)

    for r in results:
        coords=r.boxes.xyxy
        if(coords.size(dim=0) !=0):
            #all coordinates are normalized between 0-1
            xmid = int((coords[0][0] + coords[0][2])/2)/640.0
            if abs(xmid-0.5) < 0.01:
                print("CENTER")
                ser.write(b'CENTER\n')
            elif xmid + 0.01 > 0.5 or xmid - 0.01 > 0.5:
                print("RIGHT")
                ser.write(b'LEFT\n')
            elif xmid + 0.01 < 0.5 or xmid - 0.01 < 0.5 :
                print("LEFT") 
                ser.write(b'RIGHT\n')
            ymid = int((coords[0][1] + coords[0][3])/2)/480.0
            print(xmid)
        else:
            print("NO TARGETS FOUND")




    cv2.imshow("test",img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
model(source=0, conf=0.6,show=True,save=False)